In [1]:
import cv2
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output
import time

from tiki.mini import TikiMini

In [2]:
tiki = TikiMini()

In [ ]:
# Hyper Parameters
#lower_green = np.array([67, 23, 101])
#upper_green = np.array([80, 255, 168])

# Byung Ddu ggung
lower_event = np.array([90, 100, 100])
upper_event = np.array([120, 255, 255])


lower_line = np.array([0, 0, 0])
upper_line = np.array([180, 255, 90])

rho = 1
theta = np.pi / 180
threshold = 80
min_line_length = 50
max_line_gap = 10
angle_threshold = 30

In [4]:
class Wrapper:
    def __init__(self, save_name=None):
        self.cap = cv2.VideoCapture(
            "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, framerate=15/1, format=NV12 ! "
            "nvvidconv flip-method=2 ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
        )
    def __enter__(self):
        print('entered')
        return self.cap
    def __exit__(self,a,b,c):
        self.cap.release()
        print('released')

In [5]:
def detect_lines(image):
    
    crop_img = image[240:480, 200:480].copy()
    
    hsv_image = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    color_mask = cv2.inRange(hsv_image, lower_line, upper_line)
    masked_image = cv2.bitwise_and(crop_img, crop_img, mask=color_mask)
    gray_mask = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    lines, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return crop_img, lines

In [6]:
def adjust_wheel_speed(cx):
    if (cx > 160):
        if (cx > 200): # LEFT CORNER
            tiki.set_motor_power(tiki.MOTOR_LEFT, 50)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 10)
        else: # LINE
            tiki.set_motor_power(tiki.MOTOR_LEFT, 50)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
    elif (cx < 100):
        if (cx < 60): # RIGHT CORNER
            tiki.set_motor_power(tiki.MOTOR_LEFT, 10)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 50)
        else: # LINE
            tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
            tiki.set_motor_power(tiki.MOTOR_RIGHT, 50)
    else: # NORMAL
        tiki.set_motor_power(tiki.MOTOR_LEFT, 30)
        tiki.set_motor_power(tiki.MOTOR_RIGHT, 30)
    return 

In [ ]:
def detect_event_color(image, lower_event, upper_event):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_event, upper_event)
    return mask

def calculate_center_area(image, center_width=200, center_height=60):
    height, width = image.shape[:2]
    
    center_x = width // 2 
    center_y = height // 2 + 10
    start_x = center_x - center_width // 2
    start_y = center_y - center_height // 2
    end_x = start_x + center_width
    end_y = start_y + center_height
    
    center_area = image[start_y:end_y, start_x:end_x]
    
    return center_area, (start_x, start_y, end_x, end_y)

def detect_event_in_center(mask, center_coords):
    start_x, start_y, end_x, end_y = center_coords
    center_mask = mask[start_y:end_y, start_x:end_x]
    
    event_detected = np.sum(center_mask) > 0
    
    event_point = None
    if event_detected:
        contours, _ = cv2.findContours(center_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"]) + start_x
                cy = int(M["m01"] / M["m00"]) + start_y
                green_point = (cx, cy)
    
    return event_detected, green_point

In [ ]:
# 카메라에서 실시간 이미지 출력하는 함수
def main():
    video_widget = widgets.Image(format='jpeg')
    tiki.set_motor_mode(tiki.MOTOR_MODE_PID)
    
    with Wrapper() as cap:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Line
            crop_img, lines = detect_lines(frame)
            
            if len(lines) > 0:
                
                c = max(lines, key=cv2.contourArea)
                M = cv2.moments(c)
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])

                #cv2.line(crop_img,(cx,0),(cx,640),(255,0,0),1)
                #cv2.line(crop_img,(0,cy),(480,cy),(255,0,0),1)
                cv2.drawContours(crop_img, lines, -1, (0,255,0), 1)

            adjust_wheel_speed(cx)
        
            # Event Detection
            event_mask = detect_event_color(frame, lower_event, upper_event)
            center_coords = calculate_center_area(event_mask)
            event_detected, event_point = detect_event_in_center(event_mask, center_coords)
            if event_detected:
                event_detection_count += 1
            else:
                event_detection_count = 0 
            event_area = cv2.bitwise_and(frame, frame, mask=event_mask)
            alpha = 0.3
            mask = event_mask.astype(bool)
            frame[mask] = cv2.addWeighted(frame, alpha, event_area, 1 - alpha, 0)[mask]

            start_x, start_y, end_x, end_y = center_coords
            cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)

            if event_point:
                cv2.circle(frame, event_point, 10, (0, 255, 0), -1)
                cv2.putText(frame, f"Green: ({event_point[0]}, {event_point[1]})", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            frame[240:480, 200:480] = crop_img
            cv2.putText(frame, f"cx: {cx}s", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            #left_encoder = tiki.get_encoder(tiki.MOTOR_LEFT)
            #right_encoder = tiki.get_encoder(tiki.MOTOR_RIGHT)
            #cv2.putText(frame, f"L_Motor: {left_encoder}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            #cv2.putText(frame, f"R_Motor: {right_encoders}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            
            _, buffer = cv2.imencode('.jpg', frame)
            video_widget.value = buffer.tobytes()
            
            clear_output(wait=True)
            display(video_widget)
            time.sleep(0.03)  # 프레임 간 간격

if __name__ == '__main__':
    main()

released


KeyboardInterrupt: 